In [1]:
import numpy as np
import pandas as pd
from unidecode import unidecode
from nltk.metrics import jaccard_distance

---
# MATCHING THE INGREDIENTS

In [2]:
org_products = pd.read_excel('products.xlsx')
products = org_products.copy()

display(products)

,Unnamed: 0.1,Unnamed: 0,Barcode,ingredient_list_original,product_name,ingredient_list
0,0,0,3600542399326,"Aqua / Water, Aluminum Chlorohydrate, Cetearyl...",Garnier Mineral Hyaluronic Care 72H Deodorant ...,aqua / water
1,1,1,3600542399326,"Aqua / Water, Aluminum Chlorohydrate, Cetearyl...",Garnier Mineral Hyaluronic Care 72H Deodorant ...,aluminum chlorohydrate
2,2,2,3600542399326,"Aqua / Water, Aluminum Chlorohydrate, Cetearyl...",Garnier Mineral Hyaluronic Care 72H Deodorant ...,cetearyl alcohol
3,3,3,3600542399326,"Aqua / Water, Aluminum Chlorohydrate, Cetearyl...",Garnier Mineral Hyaluronic Care 72H Deodorant ...,ceteareth-33
4,4,4,3600542399326,"Aqua / Water, Aluminum Chlorohydrate, Cetearyl...",Garnier Mineral Hyaluronic Care 72H Deodorant ...,parfum / fragrance
...,...,...,...,...,...,...
228225,228234,236715,8436575091013,"2-Ethyl-3,(5 Or 6)-Dimethylpyrazine, 2-Trans, ...",NaN,"6,10-dimethylundeca-1,5,9-trien-4-ol"
228226,228235,236716,8436575091013,"2-Ethyl-3,(5 Or 6)-Dimethylpyrazine, 2-Trans, ...",NaN,6-amino-m-cresol
228227,228236,236717,8436575091013,"2-Ethyl-3,(5 Or 6)-Dimethylpyrazine, 2-Trans, ...",NaN,"6,7-dihydrolinalool"
228228,228237,236718,8436575091013,"2-Ethyl-3,(5 Or 6)-Dimethylpyrazine, 2-Trans, ...",NaN,6-hydroxyindole


In [3]:
nan_rows2 = products['ingredient_list'].isna()
print(nan_rows2.sum())
products = products.dropna(subset=['ingredient_list'])

0


In [4]:
products[products[['product_name', 'ingredient_list']].duplicated()]

,Unnamed: 0.1,Unnamed: 0,Barcode,ingredient_list_original,product_name,ingredient_list
109,109,123,4005900939067,"Butane, Isobutane, Propane, Isopropyl Palmitat...",No.16 Splashy Citrus Deodorant Spray\nStart Th...,citral
110,110,124,4005900939067,"Butane, Isobutane, Propane, Isopropyl Palmitat...",No.16 Splashy Citrus Deodorant Spray\nStart Th...,parfum
532,532,782,4011700352012,"s, ALCOHOL, WATER (AQUA), FRAGRANCE (PERFUM), ...",Woman No. 2 Eau De Toilette\nFor The Fashionab...,geraniol
536,536,786,4011700352012,"s, ALCOHOL, WATER (AQUA), FRAGRANCE (PERFUM), ...",Woman No. 2 Eau De Toilette\nFor The Fashionab...,reds
537,537,787,4011700352012,"s, ALCOHOL, WATER (AQUA), FRAGRANCE (PERFUM), ...",Woman No. 2 Eau De Toilette\nFor The Fashionab...,reds
...,...,...,...,...,...,...
227131,227140,235611,4008233150239,"I LOVE YOU,, Sodium chloride, perfume (fragra...",Gift Set For You,geraniol
227132,227141,235612,4008233150239,"I LOVE YOU,, Sodium chloride, perfume (fragra...",Gift Set For You,glycerin
227133,227142,235613,4008233150239,"I LOVE YOU,, Sodium chloride, perfume (fragra...",Gift Set For You,aqua (water)
227134,227143,235614,4008233150239,"I LOVE YOU,, Sodium chloride, perfume (fragra...",Gift Set For You,tocopherol


In [5]:
org_ingredients = pd.read_csv('ingredients.csv')
ingredients = org_ingredients.copy()

display(ingredients)

,Unnamed: 0,generic_name,synonym
0,0,Niacinamide,niacinamide
1,1,Niacinamide,vitamin b3
2,2,Niacinamide,nicotinamide
3,3,Niacinamide,3-pyridinecarboxamide
4,4,Hyaluronic Acid,hyaluronic acid
...,...,...,...
28813,28813,Parfum,perfum fragrance
28814,28814,Parfum,perfume
28815,28815,Aroma,aroma (flavor)
28816,28816,Aroma,aroma/flavor


In [6]:
nan_rows = ingredients['synonym'].isna()
print(nan_rows.sum())

0


In [7]:
# Prvicen kod
exact_matches = products['ingredient_list'][products['ingredient_list'].isin(ingredients['synonym'].values)].tolist()


In [8]:
print('Procent na pogodeni:               ', round(len(exact_matches)/len(products['ingredient_list']), 2)*100, '%')
print('Cela lista na sostojki vo produkti:', len(products['ingredient_list']))
print('Pogodeni:                          ', len(exact_matches))
print('Nepogodeni:                        ', len(products['ingredient_list']) - len(exact_matches))

Procent na pogodeni:                77.0 %
Cela lista na sostojki vo produkti: 228230
Pogodeni:                           175927
Nepogodeni:                         52303


In [9]:
# # Assuming products and ingredients are your dataframes
# matches_by_product = {}

# for product_name in products['product_name'].unique():
#     # Filter the dataframes for the specific product
#     product_ingredient_list = set(products.loc[products['product_name'] == product_name, 'ingredient_list'].astype(str))
#     ingredient_synonyms = set(ingredients['synonym'].astype(str))

#     # Find matches for the specified product
#     matches_by_product[product_name] = ingredient_synonyms.intersection(product_ingredient_list)

In [10]:
# # Print the matches for each product
# for product, matches in matches_by_product.items():
#     print('\n\nProduct name:\t', product) 
#     #print('\t\t', product)
#     print('\nIngredients:')
#     for match in matches:
#         # Find the corresponding 'generic_name' based on the index of the 'synonym'
#         generic_name = ingredients.loc[ingredients['synonym'] == match, 'generic_name'].iloc[0]
#         print('\t\t', generic_name)
#     print('\n------------------------------------------------------------------------------')

In [11]:
# Snimanje na podatocite so explode funkcija

# Assuming products and ingredients are your dataframes
not_matched_by_product_list = []

# Iterate over each product
for product_name in products['product_name'].unique():
    # Filter the dataframes for the specific product
    product_ingredient_list = set(products.loc[products['product_name'] == product_name, 'ingredient_list'].astype(str))
    ingredient_synonyms = set(ingredients['synonym'].astype(str))

    # Find ingredients not matched by product
    not_matched = product_ingredient_list - ingredient_synonyms

    # Append product_name and not_matched to the result list
    not_matched_by_product_list.append({'Product_Name': product_name, 'Not_Matched_Ingredients': list(not_matched)})

# Convert the list of dictionaries to a DataFrame
not_matched_df = pd.DataFrame(not_matched_by_product_list)

# Explode the 'Not_Matched_Ingredients' column to create new columns for each ingredient
not_matched_df = not_matched_df.explode('Not_Matched_Ingredients').reset_index(drop=True)

# Save the DataFrame to a CSV file
not_matched_df.to_csv('not_matched_ingredients.csv', index=False)

print("CSV file 'not_matched_ingredients.csv' saved successfully.")

CSV file 'not_matched_ingredients.csv' saved successfully.


In [12]:
# products['ingredient_list'] = products['ingredient_list'].str.strip()
# ingredients['synonym'] = ingredients['synonym'].str.strip()

# # Print statement for debugging
# print("After stripping whitespaces:")
# print(products['ingredient_list'].head())
# print(ingredients['synonym'].head())

# # Step 2: Convert to lowercase for case-insensitivity
# products['ingredient_list'] = products['ingredient_list'].str.lower()
# ingredients['synonym'] = ingredients['synonym'].str.lower()

# # Print statement for debugging
# print("\nAfter converting to lowercase:")
# print(products['ingredient_list'].head())
# print(ingredients['synonym'].head())

# # Step 3: Encode and decode to handle special characters
# products['ingredient_list'] = products['ingredient_list'].str.encode('utf-8').str.decode('utf-8')
# ingredients['synonym'] = ingredients['synonym'].str.encode('utf-8').str.decode('utf-8')

# # Print statement for debugging
# print("\nAfter encoding and decoding:")
# print(products['ingredient_list'].head())
# print(ingredients['synonym'].head())

# # Step 4: Print mismatches for a few products
# for product_name in products['product_name'].unique():
#     product_ingredient_list = set(products.loc[products['product_name'] == product_name, 'ingredient_list'].astype(str))
#     ingredient_synonyms = set(ingredients['synonym'].astype(str))
#     not_matched = product_ingredient_list - ingredient_synonyms
#     print(f"\nProduct: {product_name}, Not Matched: {not_matched}")

In [13]:
# # Assuming products and ingredients are your dataframes
# not_matched_by_product_list = []

# # Iterate over each product
# for product_name in products['product_name'].unique():
#     # Filter the dataframes for the specific product
#     product_ingredient_list = products.loc[products['product_name'] == product_name, 'ingredient_list'].dropna().astype(str).tolist()
#     ingredient_synonyms = ingredients['synonym'].dropna().astype(str).tolist()

#     # Find ingredients not matched by product
#     not_matched = list(set(product_ingredient_list) - set(ingredient_synonyms))

#     # Append product_name and not_matched to the result list
#     not_matched_by_product_list.append({'Product_Name': product_name, 'Not_Matched_Ingredients': not_matched})

# # Convert the list of dictionaries to a DataFrame
# not_matched_df = pd.DataFrame(not_matched_by_product_list)

# # Explode the 'Not_Matched_Ingredients' column to create new columns for each ingredient
# not_matched_df = not_matched_df.explode('Not_Matched_Ingredients').reset_index(drop=True)

# # Save the DataFrame to a CSV file
# not_matched_df.to_csv('not_matched_ingredients.csv', index=False)

# print("CSV file 'not_matched_ingredients.csv' saved successfully.")


In [14]:
org_not_matched_ingredients = pd.read_csv('not_matched_ingredients.csv')
not_matched = org_not_matched_ingredients.copy()

display(not_matched)

,Product_Name,Not_Matched_Ingredients
0,Garnier Mineral Hyaluronic Care 72H Deodorant ...,phenoxyethanol.
1,Velnea Fingering Silik.Lila,silicone
2,Echt Ki? 1/2 Lnisch Wasser - Eau De Cologne\nM...,NaN
3,Remix Cologne Orange Blossom Eau De Cologne Na...,water (aqua)
4,Remix Cologne Orange Blossom Eau De Cologne Na...,ext. violet 2 (ci 60730).
...,...,...
43054,Retinol Cream,triticum vulgare (wheat) germ oil
43055,Retinol Cream,theobroma cacao (cocoa) seed butter
43056,Retinol Cream,cucumis sativus (cucumber)fruit extract
43057,Retinol Cream,hexyl cinnamal.


In [15]:
nan_rows = not_matched['Not_Matched_Ingredients'].isna()
print(nan_rows.sum())
#not_matched = not_matched.dropna(subset=['Not_Matched_Ingredients'])

1056


In [16]:
def find_first_match(ingredient, candidates, threshold=0.35):
    ingredient_set = set(ingredient.split())

    # Set to keep track of matched ingredients
    matched_ingredients = set()

    for candidate in candidates:
        # Skip if candidate ingredient has already been matched
        if pd.notna(candidate):  # Check if candidate is not NaN
            if candidate in matched_ingredients:
                continue

            if isinstance(candidate, str):  # Check if candidate is a string
                candidate_set = set(candidate.split())
                union_size = len(ingredient_set.union(candidate_set))

                if union_size != 0:
                    distance = jaccard_distance(ingredient_set, candidate_set)
                    similarity = 1 - distance

                    if similarity > threshold:
                        # Return the first match and exit the function
                        return candidate

                        # Add the matched ingredient to the set
                        matched_ingredients.add(candidate)
    return None

In [17]:
print("Before stripping whitespaces:")
print("Products - Unique Values:", products['ingredient_list'].nunique())
print("Ingredients - Unique Values:", ingredients['synonym'].nunique())

# Step 1: Remove leading and trailing whitespaces
products['ingredient_list'] = products['ingredient_list'].str.strip()
ingredients['synonym'] = ingredients['synonym'].str.strip()

# Print the count of unique values after stripping whitespaces
print("\nAfter stripping whitespaces:")
print("Products - Unique Values:", products['ingredient_list'].nunique())
print("Ingredients - Unique Values:", ingredients['synonym'].nunique())

Before stripping whitespaces:
Products - Unique Values: 16874
Ingredients - Unique Values: 28818

After stripping whitespaces:
Products - Unique Values: 16874
Ingredients - Unique Values: 28818
